In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain.vectorstores import Chroma
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import pickle
from langchain import hub


# Document loader

In [2]:
import glob

pdf_files = glob.glob("src/*.pdf")
pages = []

for pdf_file in pdf_files:
    pages.extend(PyPDFLoader(pdf_file).load_and_split())

for page in pages:
    page.page_content = page.page_content.replace("Imprimerie Officielle de la République Tunisienne", "")


# Splitter


In [5]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
len(chunks)


28127

# Vectorstores
## for now BAAI/bge-m3 is the best embedder

#### In the cloud (pinecone)

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = "9721efa4-ff98-4bc5-9b28-93919d1657a5"
# embedder = HuggingFaceEmbeddings(
#     model_name = 'dangvantuan/sentence-camembert-large'
# )

index_name = "pfa"

#docsearch = PineconeVectorStore.from_documents(chunks, embedder, index_name=index_name)
#docsearch = PineconeVectorStore(index_name=index_name,embedding=embedder)

#### Locally (chroma)

In [19]:

embedder = HuggingFaceEmbeddings(
    model_name = "BAAI/bge-m3"
)
persist_directory = 'docs/chroma_bge/'
#vectordb.delete_collection()
# vectordb = Chroma.from_documents(
#     documents=chunks,
#     embedding=embedder,
#     persist_directory=persist_directory
# ) 
# vectordb.persist()
vectordb = Chroma(persist_directory="docs/chroma_bge/",embedding_function=embedder)
print(vectordb._collection.count())

28127


# Retrieval
### 3 methodes

In [50]:

query="quels sont les punitions de fraude?"

#méthode 1 avec retriever
# retriever = vectordb.as_retriever(search_type="mmr")
# matched_docs = retriever.get_relevant_documents(query)
# for i, d in enumerate(matched_docs):
#     print(f"\n## Document {i}\n")
#     print(d.page_content)

#méthode 2 avec mmr
found_docs = vectordb.max_marginal_relevance_search(query, k=4, fetch_k=100)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n\n")

#méthode 3 avec similarity search
# sim_docs = vectordb.similarity_search_with_score(query, k=5)  
# for result in sim_docs:
#     print("\n")
#     print(result[1])
#     print(result[0].page_content)
    

1. Article 387.- Sont passibles d'un emprisonnement de trois mois à 
un an et d'une amende comprise entre deux et trois fois la valeur des 
marchandises de fraude, de la confiscation des marchandises de 
fraude, des moyens de transport et des objets ayant  servi à masquer la 
fraude, les auteurs de délits de contrebande par une réunion de trois 
individus et plus jusqu'à six inclusiveme nt, que tous portent ou non 
des marchandises de fraude. 
C- Délits de troisième classe 


2. verser une rançon , exécuter un ordre ou une condition, en ayant recours à 
la fraude , à la violence ou à la menace de violence , ou en utilisant une arme, en portant un faux uniforme, une fausse identité, un faux ordre de 
l’autorité publiqu e ou s’il en résulte un préjudic e corporel ou un e maladie.  
La peine encourue est la peine de mort et une amende de deux cent 
mille dinars, s’il en résulte la mort. 


3. peine est porté à cinq ans de prison. 


4. écriture, pour vol, pour abus de confiance, pour escr

# Prompt Engineering and generation

In [51]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = hub.pull("mehdixlabetix/rag-law-fr")
prompt = prompt.invoke({"context":format_docs(found_docs), "question":query})
print(prompt)

text=" Tu es un assistant juridique spécialisé dans la loi en Tunisie. Ta mission est de répondre aux questions des gens sur différents aspects juridiques ,en te limitant aux informations générales et en évitant les cas sensibles ou extrêmes. Si une question dépasse ton champ d'expertise ou si elle concerne un sujet délicat, tu dois informer l'utilisateur que tu ne peux pas fournir d'aide spécifique dans ce cas.Utilise les pièces suivantes du contexte pour répondre. Utilise un langage simple et accessible pour garantir que tout le monde puisse comprendre tes réponses. Gardez votre réponse brève et efficace maximum 3 phrase.\n Question: quels sont les punitions de fraude?.\nContexte: Article 387.- Sont passibles d'un emprisonnement de trois mois à \nun an et d'une amende comprise entre deux et trois fois la valeur des \nmarchandises de fraude, de la confiscation des marchandises de \nfraude, des moyens de transport et des objets ayant  servi à masquer la \nfraude, les auteurs de délits 

In [52]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm.invoke(prompt)

"According to Article 387 of the Tunisian Penal Code, fraud can lead to penalties such as imprisonment for three months to one year and a fine equal to two to three times the value of the fraudulent goods. Additionally, the court may confiscate the goods, means of transportation, and objects used to commit the fraud, as well as impose a sentence of rançon, execution of an order or condition, or use of violence or threat of violence, resulting in corporal or moral harm. The punishment can also be death, with an fine of two hundred thousand dinars. If the fraud results in a person's death, the punishment is increased to five years in prison."